<a href="https://colab.research.google.com/github/DavidP0011/etl/blob/main/GBQ_global_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INICIALIZACIÓN

In [1]:
# @title INSTALACIÓN DE LIBRERÍAS


In [2]:
# @title IMPORTACIÓN DE LIBRERÍAS SISTEMA

import pandas as pd
import sys
import importlib

from google.colab import auth
auth.authenticate_user()
print('[google.colab authenticate_user]:\nAuthenticated!\n')

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

[google.colab authenticate_user]:
Authenticated!

Mounted at /content/drive


In [3]:
# @title GBQ_tables_fields_info_df
def GBQ_tables_fields_info_df(config):
    """
    Retorna un DataFrame con la información de datasets, tablas y campos de un proyecto de BigQuery.

    Args:
        config (dict): Diccionario de configuración con:
            - 'project_id' (str) [requerido]: El ID del proyecto de BigQuery.
            - 'datasets' (list) [opcional]: Lista de los IDs de los datasets a consultar.
              Si no se proporciona, se consultan todos los disponibles en el proyecto.
            - 'include_tables' (bool) [opcional]: Indica si se deben incluir las tablas
              en el esquema. Por defecto es True.

    Returns:
        pd.DataFrame: DataFrame con las columnas:
            [
                'project_id',
                'dataset_id',
                'table_name',
                'field_name',
                'field_type',
                'num_rows',
                'num_columns',
                'size_mb'
            ]
    """
    from google.cloud import bigquery
    from google.colab import auth
    import pandas as pd

    # 1. Autenticamos (en caso de estar en Colab)
    auth.authenticate_user()

    project_id = config.get('project_id')
    if not project_id:
        raise ValueError("El 'project_id' es un argumento requerido en el diccionario de configuración.")

    # Parámetros adicionales
    datasets_incluidos = config.get('datasets', None)
    include_tables = config.get('include_tables', True)

    # 2. Creamos el cliente de BigQuery
    client = bigquery.Client(project=project_id)

    # 3. Obtenemos la lista de datasets
    if datasets_incluidos:
        datasets = [client.get_dataset(f"{project_id}.{dataset_id}") for dataset_id in datasets_incluidos]
    else:
        datasets = list(client.list_datasets(project=project_id))

    # Estructura temporal para ir almacenando la info de tablas y campos
    tables_info_list = []

    for dataset in datasets:
        dataset_id = dataset.dataset_id
        full_dataset_id = f"{project_id}.{dataset_id}"

        # 4. Si include_tables=True, listamos las tablas de cada dataset
        if include_tables:
            tables = list(client.list_tables(full_dataset_id))
            for table_item in tables:
                table_ref = client.get_table(table_item.reference)  # Obtenemos objeto Table

                # Datos de la tabla
                table_name = table_item.table_id
                num_rows = table_ref.num_rows
                num_columns = len(table_ref.schema)
                size_mb = table_ref.num_bytes / (1024 * 1024)  # Convertir de bytes a MB

                # 5. Campos (schema) de la tabla
                fields = table_ref.schema

                # Cada campo de la tabla se agrega como un registro independiente
                if fields:
                    for field in fields:
                        tables_info_list.append({
                            'project_id': project_id,
                            'dataset_id': dataset_id,
                            'table_name': table_name,
                            'field_name': field.name,
                            'field_type': field.field_type,
                            'num_rows': num_rows,
                            'num_columns': num_columns,
                            'size_mb': round(size_mb, 2)
                        })
                else:
                    # Si la tabla no tiene campos, se ingresa un registro con field_name=None
                    tables_info_list.append({
                        'project_id': project_id,
                        'dataset_id': dataset_id,
                        'table_name': table_name,
                        'field_name': None,
                        'field_type': None,
                        'num_rows': num_rows,
                        'num_columns': num_columns,
                        'size_mb': round(size_mb, 2)
                    })
        else:
            # En caso de no incluir tablas, añadimos el dataset sin info de tablas
            # (aunque rara vez se desea esto en un DataFrame).
            tables_info_list.append({
                'project_id': project_id,
                'dataset_id': dataset_id,
                'table_name': None,
                'field_name': None,
                'field_type': None,
                'num_rows': None,
                'num_columns': None,
                'size_mb': None
            })

    # 6. Convertimos la lista de diccionarios en DataFrame
    df_tables_fields = pd.DataFrame(tables_info_list)

    return df_tables_fields

In [4]:
# @title df_to_GSheets()

import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd

def df_to_GSheets(dic):
    """
    Volca el contenido de un DataFrame en una hoja de Google Sheets.

    Parámetros:
        dic (dict): Diccionario con las siguientes claves:
            - 'df': DataFrame a volcar.
            - 'spreadsheet_id': URL completa o ID del spreadsheet de Google Sheets.
            - 'worksheet_name': Nombre de la pestaña (worksheet) destino.
            - 'json_keyfile': Ruta al archivo JSON con las credenciales.

    La función se conecta a Google Sheets, busca (o crea) la pestaña indicada,
    limpia su contenido y volca el DataFrame.
    """
    # Autenticación mediante el archivo de credenciales JSON
    gc = gspread.service_account(filename=dic['json_keyfile'])

    # Comprobar si se ha pasado la URL completa o solo el ID y abrir el spreadsheet correspondiente
    spreadsheet_ref = dic['spreadsheet_id']
    if spreadsheet_ref.startswith("http"):
        sh = gc.open_by_url(spreadsheet_ref)
    else:
        sh = gc.open_by_key(spreadsheet_ref)

    # Intentar obtener la hoja especificada; si no existe, se crea
    try:
        worksheet = sh.worksheet(dic['worksheet_name'])
    except gspread.exceptions.WorksheetNotFound:
        # Crear una nueva hoja con tamaño arbitrario (ajustar según necesidad)
        worksheet = sh.add_worksheet(title=dic['worksheet_name'], rows="100", cols="20")

    # Limpiar el contenido previo de la hoja
    worksheet.clear()

    # Volcar el DataFrame en la hoja de cálculo
    set_with_dataframe(worksheet, dic['df'])

    print("DataFrame volcado correctamente en la hoja '{}' del spreadsheet.".format(
        dic['worksheet_name']
    ))


# EJECUCIONES

In [5]:
# @title GBQ INFO GLOBAL
%%time

# Configuración
project_id = 'animum-dev-datawarehouse' # @param {"type":"string"}
datasets = [ ]                          # @param {"type":"string"}
include_tables = True                   # @param {"type":"boolean"}

config = {
    'project_id': project_id,
    'datasets': datasets,
    'include_tables': include_tables
}

# Llamada a la nueva función
full_info_from_GBQ_df = GBQ_tables_fields_info_df(config)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Visualizar

# display(full_info_from_GBQ_df)


# Restaurar configuración de pandas
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')


CPU times: user 2.12 s, sys: 210 ms, total: 2.33 s
Wall time: 1min 29s


In [6]:
# @title GBQ DATASETS
print("Datasets disponibles:\n")
for dataset in full_info_from_GBQ_df['dataset_id'].unique():
    print(f"{dataset}")


Datasets disponibles:

BOMojo_staging_01
FAds_staging_v01
GAds_staging_v01
IMDb_raw_01
IMDb_staging_01
cd2_01raw_01
facebook_ads_raw_v01
facebook_ads_raw_v01_facebook_ads
facebook_ads_raw_v01_facebook_ads_source
fivetran_metadata
fivetran_metadata_fivetran_platform
fivetran_metadata_stg_fivetran_platform
google_ads_raw_01
google_ads_raw_01_google_ads
google_ads_raw_01_google_ads_source
hubspot_BI_v01
hubspot_raw_v01
hubspot_raw_v01_hubspot
hubspot_raw_v01_stg_hubspot
hubspot_staging_v01
mkt_02st_01
tablas_mapeo
tp_02st_01
tp_03bi_01
vl_01raw_01


In [7]:
# @title SUBIDA A GOOGLE SHEETS

import pandas as pd

# Diccionario con los parámetros (usando URL completa)
parametros = {
    'df': full_info_from_GBQ_df,  # o full_info_from_GBQ_df si ese es tu DataFrame real
    'spreadsheet_id': 'https://docs.google.com/spreadsheets/d/1aJCGTJtDu_ODqBc4zUcrpQ-q6PE_HN0rO4mwYMIhCXw',
    'worksheet_name': 'DATA',
    'json_keyfile': '/content/drive/Othercomputers/Mi PC/proyectos/api_keys/animum-dev-datawarehouse-ef58845eb41a.json'
}

# Volcamos el DataFrame en la hoja de Google Sheets
df_to_GSheets(parametros)


DataFrame volcado correctamente en la hoja 'DATA' del spreadsheet.
